# Cyclone Track

We will see how we can query wave conditions from altimeter database along a specified cyclone track.

## Loading RADWave library and initialisation

We first start by importing **RADwave** library into our working space.

In [ ]:
import RADWave as rwave

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

First we will use already downloaded dataset of track of cyclone YASI. The data was obtained from the Australian Bureau of Meteorology ([BOM cyclone tracks](http://www.bom.gov.au/cyclone/history/tracks/)).

To load this file in the `waveAnalysis` class, the cyclone track needs to be a `CSV` file with in the header having the following keyword names `lon`, `lat` & `datetime`. 

To only visualise the tracks on a map, a minimal number of options have to be set as shown in the cell below:

In [ ]:
cyc = rwave.waveAnalysis(cycloneCSV='../../dataset/2010-YASI.csv')

Plotting the tracks is done by using the following function:

In [ ]:
cyc.plotCycloneTracks(title="Cyclone YASI Track", markersize=100, zoom=4, 
                     extent=[138, 180, -30, -10], fsize=(12, 10))

The geographical extent of the cyclone path and the associated time frame can be infered from the figure above. 

This was used to specify the altimeter data record **location** and **temporal extent** when using the Australian Ocean Data Network portal [AODN](https://portal.aodn.org.au/).


> As for the other examples, we recomend to look at **RADWave** [documentation](https://radwave.readthedocs.io/en/latest/usage.html#getting-altimeter-values-from-data-providers) and the embeded video that explain how to select both a spatial bounding box and a temporal extent from the portal and how to export the file containing the `List of URLs`. This `TXT` file contains a list of `NETCDF` files for each available satellites. 


We will now create 2 new **RADWave** classes names (`wa_east` and `wa_west`) that will, in addition to the cyclone track, set the list of altimeter `NETCDF` URLs files to query for the analyse in 2 different regions.

For a detail overview of the options available in this class, you can have a look at the [waveAnalysis API](https://radwave.readthedocs.io/en/latest/RADWave.html#RADWave.altiwave.waveAnalysis).


We also call the `processAltimeterData` function to query the actual dataset and store the altimeter data in each class. The description of this function is available from the [API](https://radwave.readthedocs.io/en/latest/RADWave.html#RADWave.altiwave.waveAnalysis.processingAltimeterData).

## For the eastern region

In [ ]:
wa_east = rwave.waveAnalysis(altimeterURL='../../dataset/IMOS_YASI_east.txt', bbox=[170, 175, -17, -12], 
                  stime=[2011,1,27], etime=[2011,2,4], cycloneCSV='../../dataset/2010-YASI.csv')

wa_east.processAltimeterData(altimeter_pick='all', saveCSV = 'altimeterDataE.csv')

## For the western region

In [ ]:
wa_west = rwave.waveAnalysis(altimeterURL='../../dataset/IMOS_YASI_west.txt', bbox=[156, 161, -16, -12], 
                  stime=[2011,1,27], etime=[2011,2,4], cycloneCSV='../../dataset/2010-YASI.csv')

wa_west.processAltimeterData(altimeter_pick='all', saveCSV = 'altimeterDataW.csv')

In the case where a cyclone track is given when initialising the `waveAnalysis` class, the `visualiseData` plots not only the extent of the altimeter dataset but also the associated path.

In [ ]:
wa_east.visualiseData(title="Altimeter data east side", extent=[138, 180, -30, -10.0], 
                 markersize=35, zoom=4, fsize=(12, 10), fsave=None)

wa_west.visualiseData(title="Altimeter data west side", extent=[138, 180, -30, -10.0], 
                 markersize=35, zoom=4, fsize=(12, 10), fsave=None)

# Extracting relevant wave dataset

Once the data has been loaded, the following step consist in computing the wave [parameters](https://radwave.readthedocs.io/en/latest/method.html#) by running the `generateTimeSeries` function.

This function computes both **instantaneous** and **monthly** wave variables:

+ significant wave height (m) - wh & wh_rolling
+ wave period (s)  - period & period_rolling
+ wave energy flux (kW/m)  - power & power_rolling
+ wave average energy density (J/m2)  - energy & energy_rolling
+ wave group velocity (m/s)  - speed & speed_rolling

In [ ]:
wa_east.generateTimeSeries()

wa_west.generateTimeSeries()

From the cyclone track, we find the closest processed altimeter geographical locations that have been recorded in the database (based on a `KDTree` search). 

> In addition to their coordinates, the altimeter dataset has to be recorded during a user defined time lapse close enough to the cyclone path time at each position.

This is done using the `close2Track` function that takes the following arguments:

+ `radius`, the maximum radius distance in degree between cyclone position and altimeter data coordinates [here set to 2.]
+ `dtmax`, the maximum difference in time between recorded cyclone date and picked altimeter data (hours) [here set to 6]

This function stores a Pandas dataframe in the `waveAnalysis` class called `cyclone_data` that contains the following variables:

+ altimeter significant wave height (m) - wH
+ altimeter wave period (s) - period
+ altimeter wave energy flux (kW/m) - power
+ altimeter wave average energy density (J/m2) - energy
+ altimeter wave group velocity (m/s) - speed
+ distance between altimeter coordinates and cyclone path (km) - dist
+ cyclone date (datetime) - cdate 
+ difference in time between recorded cyclone date and altimeter data for specific position (hours) - hours
+ cyclone latitude position - clat
+ cyclone longitude position - clon
+ altimeter data latitude position - lat
+ altimeter data longitude position - lon

Depending of the available altimeters dataset and the chosen `radius` and `dtmax` parameters, the Pandas dataframe `cyclone_data` will contain different number of values (or can potentially be empty).

In [ ]:
wa_east.close2Track(radius=2.,dtmax=6.)
display(wa_east.cyclone_data)

wa_west.close2Track(radius=2.,dtmax=6.)
display(wa_west.cyclone_data)

# Visualising the relevant altimeters date

**RADWave** provides a plotting function to visualise the processed wave data called `plotCycloneAltiPoint` that can be used to also provide information about wave parameters for each data point (by turning the `showinfo` flag to True as explained in the [API](https://radwave.readthedocs.io/en/latest/RADWave.html#RADWave.altiwave.waveAnalysis.cycloneAltiPoint).

An example of how to call this function is presented below:

In [ ]:
wa_west.plotCycloneAltiPoint(showinfo=False, extent=[138, 180, -18, -10], 
                 markersize=35, zoom=4, fsize=(12, 5))

In [ ]:
wa_west.plotCycloneAltiPoint(showinfo=True, extent=[138, 180, -18, -10], 
                 markersize=35, zoom=4, fsize=(12, 5))

> As already mentioned in the different examples, the class **waveAnalysis()** saves most of the processed wave data as Pandas dataframe (such as `timeseries` or `cyclone_data`) that can be used for further analysis.